# unified_planning for classical planning

This python notebook shows how to use the unified_planning library to model and solve a problem of classical planning

# Download and install the library



In [72]:
# begin of installation

In [73]:
!rm -rf unified-planning && git clone https://github.com/aiplan4eu/unified-planning && pip install unified-planning/

Cloning into 'unified-planning'...
remote: Enumerating objects: 9878, done.
remote: Counting objects: 100% (1619/1619), done.
remote: Compressing objects: 100% (323/323), done.
remote: Total 9878 (delta 1402), reused 1418 (delta 1296), pack-reused 8259
Receiving objects: 100% (9878/9878), 6.86 MiB | 12.74 MiB/s, done.
Resolving deltas: 100% (7241/7241), done.
Processing ./unified-planning
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for unified-planning: filename=unified_planning-0.2.2.209.dev1-py3-none-any.whl size=206025 sha256=f445a6c5eec269bf34e0376d2c201715ee47c37f91f9c5ec4972715ca76fc4f7
  Stored in directory: /root/.ca

# Download and install Tamer

In [74]:
!rm -rf up-tamer && git clone https://github.com/aiplan4eu/up-tamer && pip install up-tamer/

Cloning into 'up-tamer'...
remote: Enumerating objects: 438, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 438 (delta 98), reused 96 (delta 79), pack-reused 283
Receiving objects: 100% (438/438), 100.41 KiB | 842.00 KiB/s, done.
Resolving deltas: 100% (261/261), done.
Processing ./up-tamer
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for up-tamer: filename=up_tamer-0.0.1-py3-none-any.whl size=11819 sha256=9a8b2c35d527681532fc70a4f1b299cc898bd4251c3a67c1323ca7eb363539d7
  Stored in directory: /root/.cache/pip/wheels/4c/7d/54/0e6493c780d4154683fd4e9b06bfbe26

In [75]:
# end of installation

# Demo

Now that the setup is done, we start using the library to create the problem.

# Imports

Import the library and all the defined shortcuts, handy to get all the frequently used constructors and usage mode.

In [76]:
import unified_planning as up
from unified_planning.shortcuts import *

# Create the problem:


- Define UserTypes

In [77]:
MAX_CAPACITY = None
Entity = UserType('Entity')

Location = UserType('Location', Entity)
BigLocation = UserType('BigLocation', Location)
State = UserType('State', BigLocation)
Region = UserType('Region', BigLocation)
Province = UserType('Province', BigLocation)

SmallLocation = UserType('SmallLocation', Location)
HealthDistrict = UserType('HealthDistrict', SmallLocation)

Transport = UserType('Transport', Entity)
BigTransport = UserType('BigTransport', Transport)
Airplane = UserType('Airplane', BigTransport)
Truck = UserType('Truck', BigTransport)

SmallTransport = UserType('SmallTransport', Transport)
Drone = UserType('Drone', SmallTransport)

- Define fluents

In [78]:
geographically_belongs_to = Fluent('geographically_belongs_to', BoolType(), location=Location, delimiter=Location)
has_airport = Fluent('has_airport', BoolType(), location=Location)
is_at = Fluent('is_at', BoolType(), transport=Transport, position=Location)
capacity = Fluent('capacity', IntType(lower_bound=1, upper_bound=MAX_CAPACITY), transport=Transport)
n_of_vaccine_boxes = Fluent('n_of_vaccine_boxes', IntType(), object=Entity)
needed_vaccine_boxes = Fluent('needed_vaccine_boxes', IntType(), location=SmallLocation)

- Define actions

  Only the first one is commented in-deep

In [79]:
load = InstantaneousAction('load', l_from=Location, t_to=Transport)
# Get the action parameters, defined through **kwargs
l_from = load.parameter('l_from')
t_to = load.parameter('t_to')
# Specify the action preconditions
load.add_precondition(is_at(t_to, l_from))
load.add_precondition(GT(n_of_vaccine_boxes(l_from), 0))
# Specify the action effects
load.add_increase_effect(fluent=n_of_vaccine_boxes(t_to),   # The fluent to increase
    value=1,                                                # The increment value
    condition=GT(capacity(t_to), n_of_vaccine_boxes(t_to))) # The condition needed to apply the effect (capacity > n_of_vaccine_boxes) -> there is still space on the transport
load.add_decrease_effect(fluent=n_of_vaccine_boxes(l_from), # The fluent to decrease
    value=1,                                                # The decrement value
    condition=GT(capacity(t_to), n_of_vaccine_boxes(t_to))) # The condition needed to apply the effect (capacity > n_of_vaccine_boxes) -> there is still space on the transport

In [80]:
unload = InstantaneousAction('unload', l_to=Location, t_from=Transport)
l_to = unload.parameter('l_to')
t_from = unload.parameter('t_from')

unload.add_precondition(is_at(t_from, l_to))

unload.add_increase_effect(n_of_vaccine_boxes(l_to), 1, GT(n_of_vaccine_boxes(t_from), 0))
unload.add_decrease_effect(n_of_vaccine_boxes(t_from), 1, GT(n_of_vaccine_boxes(t_from), 0))

# Define a up.model.Variable, needed to create quantified expressions
superior_location = Variable('mid_location', Location)

move_airplane = InstantaneousAction('move_airplane', plane=Airplane, l_from=BigLocation, l_to=BigLocation)
plane = move_airplane.parameter('plane')
l_from = move_airplane.parameter('l_from')
l_to = move_airplane.parameter('l_to')

move_airplane.add_precondition(has_airport(l_from))
move_airplane.add_precondition(has_airport(l_to))
move_airplane.add_precondition(is_at(plane, l_from))
move_airplane.add_precondition(Not(is_at(plane, l_to)))
move_airplane.add_precondition(Or(geographically_belongs_to(l_from, l_to),
                            geographically_belongs_to(l_to, l_from),
                            Exists(And(geographically_belongs_to(l_from, superior_location),
                                    geographically_belongs_to(l_to, superior_location)),
                            superior_location)))

move_airplane.add_effect(is_at(plane, l_from), False)
move_airplane.add_effect(is_at(plane, l_to), True)

move_truck = InstantaneousAction('move_truck', truck=Truck, l_from=BigLocation, l_to=BigLocation)
truck = move_truck.parameter('truck')
l_from = move_truck.parameter('l_from')
l_to = move_truck.parameter('l_to')

move_truck.add_precondition(is_at(truck, l_from))
move_truck.add_precondition(Not(is_at(truck, l_to)))
move_truck.add_precondition(Or(geographically_belongs_to(l_from, l_to),
                        geographically_belongs_to(l_to, l_from),
                        Exists(And(geographically_belongs_to(l_from, superior_location),
                                geographically_belongs_to(l_to, superior_location)),
                        superior_location)))

move_truck.add_effect(is_at(truck, l_from), False)
move_truck.add_effect(is_at(truck, l_to), True)

send_drone = InstantaneousAction('send_drone', drone=Drone, l_from=Province, l_to=HealthDistrict)
drone = send_drone.parameter('drone')
l_from = send_drone.parameter('l_from')
l_to = send_drone.parameter('l_to')

send_drone.add_precondition(is_at(drone, l_from))
send_drone.add_precondition(Not(is_at(drone, l_to)))
send_drone.add_precondition(geographically_belongs_to(l_to, l_from))

send_drone.add_effect(is_at(drone, l_from), False)
send_drone.add_effect(is_at(drone, l_to), True)

retrieve_drone = InstantaneousAction('retrieve_drone', drone=Drone, l_from=HealthDistrict, l_to=Province)
drone = retrieve_drone.parameter('drone')
l_from = retrieve_drone.parameter('l_from')
l_to = retrieve_drone.parameter('l_to')

retrieve_drone.add_precondition(is_at(drone, l_from))
retrieve_drone.add_precondition(Not(is_at(drone, l_to)))
retrieve_drone.add_precondition(geographically_belongs_to(l_from, l_to))

retrieve_drone.add_effect(is_at(drone, l_from), False)
retrieve_drone.add_effect(is_at(drone, l_to), True)


- Define the problem and add fluents. The defaut_initial_value argument is used to avoid the verbosity of specifying for every possible application of that fluent his inital value (like for geographically_belongs, that takes 2 locations, we avoid writing it for every possible couple of Locations, but as default it is False, and we only set it at True for the couple of Locations that we want this fluent to be True. How to do this will be seen after the Objects are defined and added to the Problem).

In [81]:
problem = Problem('vaccine_boxes_distribution')
problem.add_fluent(geographically_belongs_to, default_initial_value=False)
problem.add_fluent(has_airport, default_initial_value=False)
problem.add_fluent(is_at, default_initial_value=False)
problem.add_fluent(capacity)
problem.add_fluent(n_of_vaccine_boxes, default_initial_value=0)
problem.add_fluent(needed_vaccine_boxes, default_initial_value=3)

- Add actions to the problem

In [82]:
problem.add_action(load)
problem.add_action(unload)
problem.add_action(move_airplane)
problem.add_action(move_truck)
problem.add_action(send_drone)
problem.add_action(retrieve_drone)

- Add the problem goal

In [83]:
goal_var = Variable('goal_var', SmallLocation)
problem.add_goal(Forall(GE(n_of_vaccine_boxes(goal_var), needed_vaccine_boxes(goal_var)), goal_var))

- Define state geography through the up.model.Object and add them to the problem.



In [84]:
italy = Object('italy', State)
problem.set_initial_value(n_of_vaccine_boxes(italy), 500)

trentino_alto_adige = Object('trentino_alto_adige', Region)
lombardia = Object('lombardia', Region)
veneto = Object('veneto', Region)

trento = Object('trento', Province)
bolzano = Object('bolzano', Province)

milano = Object('milano', Province)
bergamo = Object('bergamo', Province)

verona = Object('verona', Province)
venezia = Object('venezia', Province)

gardolo = Object('gardolo', HealthDistrict)
ravina = Object('ravina', HealthDistrict)
mattarello = Object('mattarello', HealthDistrict)

laives = Object('laives', HealthDistrict)
san_genesio = Object('san_genesio', HealthDistrict)

rho = Object('rho', HealthDistrict)
rozzano = Object('rozzano', HealthDistrict)

seriate = Object('seriate', HealthDistrict)
curno = Object('curno', HealthDistrict)

marzana = Object('marzana', HealthDistrict)
rizza = Object('rizza', HealthDistrict)

murano = Object('murano', HealthDistrict)
cavallino = Object('cavallino', HealthDistrict)

problem.add_objects([italy, trentino_alto_adige, lombardia, veneto, trento, bolzano, milano, bergamo,
verona, venezia, gardolo, ravina, mattarello, laives, san_genesio, rho, rozzano, seriate,
curno, marzana, rizza, murano, cavallino])

- Give a meaning to the state geography through the fluents

In [85]:
geographically_belongs_to = Fluent('geographically_belongs_to', BoolType(), location=Location, delimiter=Location)
has_airport = Fluent('has_airport', BoolType(), location=Location)
is_at = Fluent('is_at', BoolType(), transport=Transport, position=Location)

problem.set_initial_value(geographically_belongs_to(trentino_alto_adige, italy), True)
problem.set_initial_value(geographically_belongs_to(lombardia, italy), True)
problem.set_initial_value(geographically_belongs_to(veneto, italy), True)
problem.set_initial_value(has_airport(italy), True)

problem.set_initial_value(geographically_belongs_to(trento, trentino_alto_adige), True)
problem.set_initial_value(geographically_belongs_to(bolzano, trentino_alto_adige), True)

problem.set_initial_value(geographically_belongs_to(gardolo, trentino_alto_adige), True)
problem.set_initial_value(geographically_belongs_to(ravina, trentino_alto_adige), True)
problem.set_initial_value(geographically_belongs_to(mattarello, trentino_alto_adige), True)

problem.set_initial_value(geographically_belongs_to(laives, bolzano), True)
problem.set_initial_value(geographically_belongs_to(san_genesio, bolzano), True)

problem.set_initial_value(geographically_belongs_to(milano, lombardia), True)
problem.set_initial_value(geographically_belongs_to(bergamo, lombardia), True)
problem.set_initial_value(has_airport(lombardia), True)

problem.set_initial_value(geographically_belongs_to(rho, milano), True)
problem.set_initial_value(geographically_belongs_to(rozzano, milano), True)
problem.set_initial_value(has_airport(milano), True)

problem.set_initial_value(geographically_belongs_to(seriate, bergamo), True)
problem.set_initial_value(geographically_belongs_to(curno, bergamo), True)
problem.set_initial_value(has_airport(bergamo), True)

problem.set_initial_value(geographically_belongs_to(verona, veneto), True)
problem.set_initial_value(geographically_belongs_to(venezia, veneto), True)
problem.set_initial_value(has_airport(veneto), True)

problem.set_initial_value(geographically_belongs_to(marzana, verona), True)
problem.set_initial_value(geographically_belongs_to(rizza, verona), True)
problem.set_initial_value(has_airport(verona), True)

problem.set_initial_value(geographically_belongs_to(murano, venezia), True)
problem.set_initial_value(geographically_belongs_to(cavallino, venezia), True)

- Define airplanes, add them to the problem and give them the right inital values

In [86]:
lufthansa = Object('lufthansa', Airplane)
problem.add_object(lufthansa)
problem.set_initial_value(capacity(lufthansa), 20)
problem.set_initial_value(is_at(lufthansa, italy), True)
ryanair_1 = Object('ryanair_1', Airplane)
problem.add_object(ryanair_1)
problem.set_initial_value(capacity(ryanair_1), 10)
problem.set_initial_value(is_at(ryanair_1, italy), True)
ryanair_2 = Object('ryanair_2', Airplane)
problem.add_object(ryanair_2)
problem.set_initial_value(capacity(ryanair_2), 5)
problem.set_initial_value(is_at(ryanair_2, italy), True)

- Define trucks, add them to the problem and give them the right inital values

In [87]:
scania_1 = Object('scania_1', Truck)
problem.add_object(scania_1)
problem.set_initial_value(capacity(scania_1), 3)
problem.set_initial_value(is_at(scania_1, italy), True)
scania_2 = Object('scania_2', Truck)
problem.add_object(scania_2)
problem.set_initial_value(capacity(scania_2), 3)
problem.set_initial_value(is_at(scania_2, italy), True)
scania_3 = Object('scania_3', Truck)
problem.add_object(scania_3)
problem.set_initial_value(capacity(scania_3), 3)
problem.set_initial_value(is_at(scania_3, italy), True)
daf_1 = Object('daf_1', Truck)
problem.add_object(daf_1)
problem.set_initial_value(capacity(daf_1), 2)
problem.set_initial_value(is_at(daf_1, italy), True)
daf_2 = Object('daf_2', Truck)
problem.add_object(daf_2)
problem.set_initial_value(capacity(daf_2), 2)
problem.set_initial_value(is_at(daf_2, italy), True)
daf_3 = Object('daf_3', Truck)
problem.add_object(daf_3)
problem.set_initial_value(capacity(daf_3), 2)
problem.set_initial_value(is_at(daf_3, italy), True)
daf_4 = Object('daf_4', Truck)
problem.add_object(daf_4)
problem.set_initial_value(capacity(daf_4), 2)
problem.set_initial_value(is_at(daf_4, italy), True)
daf_5 = Object('daf_5', Truck)
problem.add_object(daf_5)
problem.set_initial_value(capacity(daf_5), 2)
problem.set_initial_value(is_at(daf_5, italy), True)

- Define drones, add them to the problem and give them the right inital values

In [88]:
drone_trento = Object('drone_trento', Drone)
problem.add_object(drone_trento)
problem.set_initial_value(capacity(drone_trento), 1)
problem.set_initial_value(is_at(drone_trento, trento), True)
drone_bolzano = Object('drone_bolzano', Drone)
problem.add_object(drone_bolzano)
problem.set_initial_value(capacity(drone_bolzano), 1)
problem.set_initial_value(is_at(drone_bolzano, bolzano), True)
drone_milano = Object('drone_milano', Drone)
problem.add_object(drone_milano)
problem.set_initial_value(capacity(drone_milano), 1)
problem.set_initial_value(is_at(drone_milano, milano), True)
drone_bergamo = Object('drone_bergamo', Drone)
problem.add_object(drone_bergamo)
problem.set_initial_value(capacity(drone_bergamo), 1)
problem.set_initial_value(is_at(drone_bergamo, bergamo), True)
drone_verona = Object('drone_verona', Drone)
problem.add_object(drone_verona)
problem.set_initial_value(capacity(drone_verona), 1)
problem.set_initial_value(is_at(drone_verona, verona), True)
drone_venezia = Object('drone_venezia', Drone)
problem.add_object(drone_venezia)
problem.set_initial_value(capacity(drone_venezia), 1)
problem.set_initial_value(is_at(drone_venezia, venezia), True)

# Problem defined, time to use Transformers

The transformers - such as the ConditionalEffectsRemover - take an instance of the problem with a specific feature - such as conditional effects - and transform a up.model.Problem with the specific feature in an equivalent up.model.Problem without said feature.

They are pretty easy to use and share the same interface.

#Conditional Effects Remover

In [89]:
cond_eff_rem = up.transformers.ConditionalEffectsRemover(problem) # Define the Transformer
unconditional_problem = cond_eff_rem.get_rewritten_problem() # Rewrite the problem
load = problem.action('load') # Retrieve an action we know to have conditional effects and print it
print('Action with conditional effects:')
print(load)
for i, unconditional_action in enumerate(cond_eff_rem.get_transformed_actions(load)): # Iterate on the actions that the transformer created by the load action.
  print(f'Derived action number {str(i+1)}:')
  print(unconditional_action)

Action with conditional effects:
action load(Location - Entity l_from, Transport - Entity t_to) {
    preconditions = [
      is_at(t_to, l_from)
      (0 < n_of_vaccine_boxes(l_from))
    ]
    effects = [
      if (n_of_vaccine_boxes(t_to) < capacity(t_to)) then n_of_vaccine_boxes(t_to) += 1
      if (n_of_vaccine_boxes(t_to) < capacity(t_to)) then n_of_vaccine_boxes(l_from) -= 1
    ]
    simulated effect = None
  }
Derived action number 1:
action cerm_load_0(Location - Entity l_from, Transport - Entity t_to) {
    preconditions = [
      is_at(t_to, l_from)
      (0 < n_of_vaccine_boxes(l_from))
      (n_of_vaccine_boxes(t_to) < capacity(t_to))
    ]
    effects = [
      n_of_vaccine_boxes(t_to) += 1
      n_of_vaccine_boxes(l_from) -= 1
    ]
    simulated effect = None
  }


#Quantifiers Remover

In [90]:
quant_rem = up.transformers.QuantifiersRemover(problem) # Define the Transformer
transformed_problem = quant_rem.get_rewritten_problem() # Rewrite the problem
move_airplane = problem.action('move_airplane') # Retrieve an action we know to have quantifiers and print it
print('Action with quantifiers:')
print(move_airplane)
for i, transformed_action in enumerate(quant_rem.get_transformed_actions(move_airplane)): # Iterate on the actions that the transformer created by the move_airplane action, we expect only one result.
  print(f'Derived action number {str(i+1)}:')
  print(transformed_action)

Action with quantifiers:
action move_airplane(Airplane - BigTransport plane, BigLocation - Location l_from, BigLocation - Location l_to) {
    preconditions = [
      has_airport(l_from)
      has_airport(l_to)
      is_at(plane, l_from)
      (not is_at(plane, l_to))
      (geographically_belongs_to(l_from, l_to) or geographically_belongs_to(l_to, l_from) or Exists (Location - Entity mid_location) (geographically_belongs_to(l_from, mid_location) and geographically_belongs_to(l_to, mid_location)))
    ]
    effects = [
      is_at(plane, l_from) := false
      is_at(plane, l_to) := true
    ]
    simulated effect = None
  }
Derived action number 1:
action qurm_move_airplane_0(Airplane - BigTransport plane, BigLocation - Location l_from, BigLocation - Location l_to) {
    preconditions = [
      has_airport(l_from)
      has_airport(l_to)
      is_at(plane, l_from)
      (not is_at(plane, l_to))
      (geographically_belongs_to(l_from, l_to) or geographically_belongs_to(l_to, l_from) or

In the printing above, we see how the precondition 

```
Exists (Location - Entity mid_location) (geographically_belongs_to(l_from, mid_location) and geographically_belongs_to(l_to, mid_location))
```

Became his instantiated version over all the objects of `Location` type.

Here the long version: 


```
((geographically_belongs_to(l_from, italy) and geographically_belongs_to(l_to, italy)) or (geographically_belongs_to(l_from, trentino_alto_adige) and geographically_belongs_to(l_to, trentino_alto_adige)) or (geographically_belongs_to(l_from, lombardia) and geographically_belongs_to(l_to, lombardia)) or (geographically_belongs_to(l_from, veneto) and geographically_belongs_to(l_to, veneto)) or (geographically_belongs_to(l_from, trento) and geographically_belongs_to(l_to, trento)) or (geographically_belongs_to(l_from, bolzano) and geographically_belongs_to(l_to, bolzano)) or (geographically_belongs_to(l_from, milano) and geographically_belongs_to(l_to, milano)) or (geographically_belongs_to(l_from, bergamo) and geographically_belongs_to(l_to, bergamo)) or (geographically_belongs_to(l_from, verona) and geographically_belongs_to(l_to, verona)) or (geographically_belongs_to(l_from, venezia) and geographically_belongs_to(l_to, venezia)) or (geographically_belongs_to(l_from, gardolo) and geographically_belongs_to(l_to, gardolo)) or (geographically_belongs_to(l_from, ravina) and geographically_belongs_to(l_to, ravina)) or (geographically_belongs_to(l_from, mattarello) and geographically_belongs_to(l_to, mattarello)) or (geographically_belongs_to(l_from, laives) and geographically_belongs_to(l_to, laives)) or (geographically_belongs_to(l_from, san_genesio) and geographically_belongs_to(l_to, san_genesio)) or (geographically_belongs_to(l_from, rho) and geographically_belongs_to(l_to, rho)) or (geographically_belongs_to(l_from, rozzano) and geographically_belongs_to(l_to, rozzano)) or (geographically_belongs_to(l_from, seriate) and geographically_belongs_to(l_to, seriate)) or (geographically_belongs_to(l_from, curno) and geographically_belongs_to(l_to, curno)) or (geographically_belongs_to(l_from, marzana) and geographically_belongs_to(l_to, marzana)) or (geographically_belongs_to(l_from, rizza) and geographically_belongs_to(l_to, rizza)) or (geographically_belongs_to(l_from, murano) and geographically_belongs_to(l_to, murano)) or (geographically_belongs_to(l_from, cavallino) and geographically_belongs_to(l_to, cavallino)))
```






Also the goal is modified, the `Forall` is removed and is replaced with an equivalent `And`; important to note that the `And` and the `Forall` are equivalent only in this particular problem.



In [91]:
print("Original goal:")
print(problem.goals[0])
print("Goal without quantifiers:")
print(transformed_problem.goals[0])

Original goal:
Forall (SmallLocation - Location goal_var) (needed_vaccine_boxes(goal_var) <= n_of_vaccine_boxes(goal_var))
Goal without quantifiers:
((needed_vaccine_boxes(gardolo) <= n_of_vaccine_boxes(gardolo)) and (needed_vaccine_boxes(ravina) <= n_of_vaccine_boxes(ravina)) and (needed_vaccine_boxes(mattarello) <= n_of_vaccine_boxes(mattarello)) and (needed_vaccine_boxes(laives) <= n_of_vaccine_boxes(laives)) and (needed_vaccine_boxes(san_genesio) <= n_of_vaccine_boxes(san_genesio)) and (needed_vaccine_boxes(rho) <= n_of_vaccine_boxes(rho)) and (needed_vaccine_boxes(rozzano) <= n_of_vaccine_boxes(rozzano)) and (needed_vaccine_boxes(seriate) <= n_of_vaccine_boxes(seriate)) and (needed_vaccine_boxes(curno) <= n_of_vaccine_boxes(curno)) and (needed_vaccine_boxes(marzana) <= n_of_vaccine_boxes(marzana)) and (needed_vaccine_boxes(rizza) <= n_of_vaccine_boxes(rizza)) and (needed_vaccine_boxes(murano) <= n_of_vaccine_boxes(murano)) and (needed_vaccine_boxes(cavallino) <= n_of_vaccine_boxes

# Disjunctive Conditions Remover



In [92]:
disj_rem = up.transformers.DisjunctiveConditionsRemover(problem) # Define the Transformer
disj_problem = disj_rem.get_rewritten_problem() # Rewrite the problem
move_truck = problem.action('move_truck') # Retrieve an action we know to have quantifiers and print it
print('Action with disjunctive preconditions:')
print(move_truck)
for i, transformed_action in enumerate(disj_rem.get_transformed_actions(move_truck)): # Iterate on the actions that the transformer created by the move_truck action, we expect 3 results.
  print(f'Derived action number {str(i+1)}:')
  print(transformed_action)

Action with disjunctive preconditions:
action move_truck(Truck - BigTransport truck, BigLocation - Location l_from, BigLocation - Location l_to) {
    preconditions = [
      is_at(truck, l_from)
      (not is_at(truck, l_to))
      (geographically_belongs_to(l_from, l_to) or geographically_belongs_to(l_to, l_from) or Exists (Location - Entity mid_location) (geographically_belongs_to(l_from, mid_location) and geographically_belongs_to(l_to, mid_location)))
    ]
    effects = [
      is_at(truck, l_from) := false
      is_at(truck, l_to) := true
    ]
    simulated effect = None
  }
Derived action number 1:
action djrm_move_truck_0(Truck - BigTransport truck, BigLocation - Location l_from, BigLocation - Location l_to) {
    preconditions = [
      is_at(truck, l_from)
      (not is_at(truck, l_to))
      geographically_belongs_to(l_from, l_to)
    ]
    effects = [
      is_at(truck, l_from) := false
      is_at(truck, l_to) := true
    ]
    simulated effect = None
  }
Derived action 

# Negative conditions remover

The Negative Conditions Remover, in order to be used, needs the expressions to be in NNF, so instad of using the Negative Conditions Remover on the original problem, we apply a pipeline of the 3 removers shown before and then we apply it.

In [93]:
cer = up.transformers.ConditionalEffectsRemover(problem, '') # The second argument is the name of the Remover. The name is applied in front of every action, so in a pipeline is better to make it ''.
qur = up.transformers.QuantifiersRemover(cer.get_rewritten_problem(), '')
dcr = up.transformers.DisjunctiveConditionsRemover(qur.get_rewritten_problem(), '')
ncr = up.transformers.NegativeConditionsRemover(dcr.get_rewritten_problem(), '')

final_problem = ncr.get_rewritten_problem()

print("Original problem kind:")
print(problem.kind)
print("\n\nTransformed problem kind:")
print(final_problem.kind)

Original problem kind:
NUMBERS: ['DISCRETE_NUMBERS']
CONDITIONS_KIND: ['EXISTENTIAL_CONDITIONS', 'NEGATIVE_CONDITIONS', 'DISJUNCTIVE_CONDITIONS', 'UNIVERSAL_CONDITIONS']
EFFECTS_KIND: ['CONDITIONAL_EFFECTS']
TYPING: ['HIERARCHICAL_TYPING', 'FLAT_TYPING']
FLUENTS_TYPE: ['NUMERIC_FLUENTS']


Transformed problem kind:
NUMBERS: ['DISCRETE_NUMBERS']
EFFECTS_KIND: ['DECREASE_EFFECTS', 'INCREASE_EFFECTS']
TYPING: ['HIERARCHICAL_TYPING', 'FLAT_TYPING']
FLUENTS_TYPE: ['NUMERIC_FLUENTS']


As we see from the problem kind, the transformers removed all the `CONDITIONS_KIND` and the `CONDITIONAL_EFFECTS` kind.

